### Load price history from the last EMA date, with ema of that date as well

In [73]:
"""
For some common calculation like EMA
"""

import pandas as pd

""" calculate EMA from a given price list, will roll the first SMA
"""
def xeam(x, period):

    # find the first non-NaN postion
    first_nn = x.index.get_loc(x.first_valid_index())
    
    # roll the first x days for fist SMA start point, roll for (period + first_nn rows) with windows (period)
    ema_rolling = x[:period + first_nn].rolling(period)

    # combine the first SMA with the rest close adj price
    ema_con = pd.concat([ema_rolling.mean(), x[period + first_nn:]])
    #ema_con.head(30)
    #df['ema_con'] = ema_con
    ema = ema_con.ewm(span=period, adjust=False).mean()

    return ema


""" 
calculate EMA from a given price list, with a given exists ema
in this case, the last given ema will be used for calculating and do NOT need roll the first SMA
"""
def cema(x, period):

    # calculate ema from the first value directly, which supposed to be an EMA value already
    # the rest of value should be adjusted close price

    ema = x.ewm(span=period, adjust=False).mean()

    return ema



In [46]:
import mysql.connector
import pandas as pd
import numpy as np

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()

    query = "SELECT price.symbol, price.effective_date, price.adj_close, ema.period, ema.ema as ema FROM \
            (SELECT * FROM (SELECT symbol, effective_date, `close`, `adj_close` FROM `market.stock_price` WHERE symbol = '{}' \
            ORDER BY effective_date DESC LIMIT 100) stock) price \
            LEFT JOIN `idc.ema` ema \
            ON ema.`symbol` = price.symbol AND ema.`effective_date` = price.effective_date \
            AND ema.`period` = 8 and ema.effective_date < '2017-10-10' \
            ORDER BY price.effective_date".format(symbol)

    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    return dfsql

In [47]:
df = load_stockdata('INTC')
df.head(20)
#df.to_csv("GPC.csv")

,symbol,adj_close,period,ema
effective_date,,,,
2017-09-29,INTC,37.638656,8.0,37.043200
2017-10-02,INTC,38.587529,8.0,37.386385
2017-10-03,INTC,38.923589,8.0,37.727986
2017-10-04,INTC,38.884053,8.0,37.984889
2017-10-05,INTC,39.071850,8.0,38.226436
2017-10-06,INTC,39.170691,8.0,38.436271
2017-10-09,INTC,39.398026,8.0,38.649994
2017-10-10,INTC,39.190460,NaN,NaN
2017-10-11,INTC,38.844516,NaN,NaN


### check exists ema and calculate differently

In [79]:
period = 8
fullcalculation = True
if ('ema' in df.columns):
    # if there is value then it will start be NOT a full run
    if not (df.ema.isnull().all()):
        # start split the ema column and merge to a new source series
        last_pos = df.index.get_loc(df.ema.last_valid_index())
        exist_ema = df.ema[:last_pos]   # right side is not included
        
        merge_price = ema_con = pd.concat([df.ema[last_pos:last_pos+1], df.adj_close[last_pos+1:]])
        
        cal_ema = cema(merge_price, period)
        df['ema'] = pd.concat([exist_ema, cal_ema])
        
        #fullcalculation = False
#merge_price
cal_ema

#print(fullcalculation)
# it's not decent structure but easier to handle the variables
#if (fullcalculation):
    # no ema and will conduct a full calculation
#    df['ema'] = xeam(df.adj_close, period)

df.head(20)

,symbol,adj_close,period,ema
effective_date,,,,
2017-09-29,INTC,37.638656,8.0,37.043200
2017-10-02,INTC,38.587529,8.0,37.386385
2017-10-03,INTC,38.923589,8.0,37.727986
2017-10-04,INTC,38.884053,8.0,37.984889
2017-10-05,INTC,39.071850,8.0,38.226436
2017-10-06,INTC,39.170691,8.0,38.436271
2017-10-09,INTC,39.398026,8.0,38.649994
2017-10-10,INTC,39.190460,NaN,38.770098
2017-10-11,INTC,38.844516,NaN,38.786635


In [82]:
x = df.adj_close
e = df.ema
# start split the ema column and merge to a new source series
# find the last ema
last_pos = x.index.get_loc(e.last_valid_index())
exist_ema = e[:last_pos]   # right side is not included

merge_price = pd.concat([e[last_pos:last_pos+1], x[last_pos+1:]])
cal_ema = merge_price.ewm(span=period, adjust=False).mean()

# calculate ema from the first value directly, which supposed to be an EMA value already
# the rest of value should be adjusted close price
#cal_ema = cema(merge_price, period)
ema = pd.concat([exist_ema, cal_ema])
ema



effective_date
2017-09-29    37.043200
2017-10-02    37.386385
2017-10-03    37.727986
2017-10-04    37.984889
2017-10-05    38.226436
2017-10-06    38.436271
2017-10-09    38.649994
2017-10-10    38.770098
2017-10-11    38.786635
2017-10-12    38.775336
2017-10-13    38.871979
2017-10-16    38.966914
2017-10-17    39.047341
2017-10-18    39.210933
2017-10-19    39.303028
2017-10-20    39.449337
2017-10-23    39.650992
2017-10-24    39.834192
2017-10-25    39.939341
2017-10-26    40.146323
2017-10-27    40.977230
2017-10-30    41.616903
2017-10-31    42.360430
2017-11-01    43.206697
2017-11-02    43.950568
2017-11-03    44.362202
2017-11-06    44.821399
2017-11-07    45.256644
2017-11-09    45.488501
2017-11-10    45.508834
                ...    
2018-01-11    44.173715
2018-01-12    43.959556
2018-01-16    43.777433
2018-01-17    43.913559
2018-01-18    44.039435
2018-01-19    44.212894
2018-01-22    44.554473
2018-01-23    44.889034
2018-01-24    45.024804
2018-01-25    45.085959
2

### Append rolling columns

In [5]:
rolling = df["adj_close"].rolling(20)
df2 = pd.DataFrame()
df2['rolling'] = rolling.sum()
df2['sma20'] = rolling.mean()
df2['stdev'] = rolling.std(ddof=0)
df2['bollinger_bands_upper'] = df2['sma20'] + df2['stdev']
df2['bollinger_bands_lower'] = df2['sma20'] - df2['stdev']

df2.where((pd.notnull(df2)),None)
df2.head(30)


,rolling,sma20,stdev,bollinger_bands_upper,bollinger_bands_lower
effective_date,,,,,
2014-02-25,NaN,NaN,NaN,NaN,NaN
2014-02-26,NaN,NaN,NaN,NaN,NaN
2014-02-27,NaN,NaN,NaN,NaN,NaN
2014-02-28,NaN,NaN,NaN,NaN,NaN
2014-03-03,NaN,NaN,NaN,NaN,NaN
2014-03-04,NaN,NaN,NaN,NaN,NaN
2014-03-05,NaN,NaN,NaN,NaN,NaN
2014-03-06,NaN,NaN,NaN,NaN,NaN
2014-03-07,NaN,NaN,NaN,NaN,NaN


In [6]:
idcdic = df2.to_dict(orient='index')

# insert from the list
for date, row in idcdic.items():
    print(date, row)
    break


2014-02-25 {'rolling': nan, 'sma20': nan, 'stdev': nan, 'bollinger_bands_upper': nan, 'bollinger_bands_lower': nan}


## EWMA & ATR

In [18]:
# get the first SMA for the rolling windows
period = 20
rolling = df[:period]["adj_close"].rolling(period)
#rolling.mean()

# combine the first SMA with the rest close adj price
con = pd.concat([rolling.mean(), df[period:]['adj_close']])
#con.head(30)

dfewma = pd.DataFrame()
dfewma['adj_close'] = df['adj_close']
dfewma['concat'] = con
dfewma['ewma20'] = con.ewm(span=period, adjust=False).mean()
#dfewma['stdev'] = rolling.std(ddof=0)
#dfewma['bollinger_bands_upper'] = df2['sma20'] + df2['stdev']
#dfewma['bollinger_bands_lower'] = df2['sma20'] - df2['stdev']

dfewma = dfewma.where((pd.notnull(dfewma)),None)
dfewma.head(30)


,adj_close,concat,ewma20
effective_date,,,
2014-02-25,34.1292,None,None
2014-02-26,34.0655,None,None
2014-02-27,34.4201,None,None
2014-02-28,34.8292,None,None
2014-03-03,34.3474,None,None
2014-03-04,34.9201,None,None
2014-03-05,34.6474,None,None
2014-03-06,34.6838,None,None
2014-03-07,34.4565,None,None


# see another notebook for ATR